## user cols

In [6]:
user_cols = [ 'user_neg_mean',
 'user_neu_mean',
 'user_pos_mean',
 'user_comp_mean',
 'user_arts_&_culture_mean',
 'user_business_&_entrepreneurs_mean',
 'user_celebrity_&_pop_culture_mean',
 'user_diaries_&_daily_life_mean',
 'user_family_mean',
 'user_fashion_&_style_mean',
 'user_film_tv_&_video_mean',
 'user_fitness_&_health_mean',
 'user_food_&_dining_mean',
 'user_gaming_mean',
 'user_learning_&_educational_mean',
 'user_music_mean',
 'user_news_&_social_concern_mean',
 'user_other_hobbies_mean',
 'user_relationships_mean',
 'user_science_&_technology_mean',
 'user_sports_mean',
 'user_travel_&_adventure_mean',
 'user_youth_&_student_life_mean',
 'user_hateful_mean',
 'user_targeted_mean',
 'user_aggressive_mean',
 'user_joy_mean',
 'user_sadness_mean',
 'user_others_mean',
 'user_anger_mean',
 'user_surprise_mean',
 'user_disgust_mean',
 'user_fear_mean',
 'user_topic_mode',
 'user_negative_count',
 'user_neutral_count',
 'user_positive_count',
 'user_arts_&_culture_count',
 'user_business_&_entrepreneurs_count',
 'user_celebrity_&_pop_culture_count',
 'user_diaries_&_daily_life_count',
 'user_family_count',
 'user_fashion_&_style_count',
 'user_film_tv_&_video_count',
 'user_fitness_&_health_count',
 'user_food_&_dining_count',
 'user_gaming_count',
 'user_learning_&_educational_count',
 'user_music_count',
 'user_news_&_social_concern_count',
 'user_other_hobbies_count',
 'user_relationships_count',
 'user_science_&_technology_count',
 'user_sports_count',
 'user_travel_&_adventure_count',
 'user_youth_&_student_life_count',
 'user_politeness_mean',
 'user_polarity_mean',
 'user_subjectivity_mean',
 'num_male','num_female']

## main func

In [1]:
import pandas as pd 
import pickle
import json
import os
import os.path
import numpy as np
import seaborn as sns
from collections import defaultdict
import joblib
import matplotlib.pyplot as plt
from matplotlib.ticker import (
                               FormatStrFormatter, 
                               AutoMinorLocator,
                               FuncFormatter,
                               )
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
%matplotlib inline
from scipy.spatial import distance
import math


import os
os.chdir('G:\My Drive\MSc_project\.MAIN')


c:\Users\berke\anaconda3\envs\env-pytorch\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\berke\anaconda3\envs\env-pytorch\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\berke\anaconda3\envs\env-pytorch\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [19]:

hashtags = ['avengers','blm','brexit','climatechange','covid','gaza','loveisland','monkeypox','nhs','olivianewtonjohn','supercup','UkraineWar']


for h_i,hashtag in enumerate(hashtags):

    cats = ['sentiment', 'topic', 'topic_single', 'irony', 'emoji', 'offensive','gender', 'user_sentiment_mode']

    df = pd.read_csv(f'multisource_analysis/user_ft_data/{hashtag}_scored_tweets.csv')
    df.drop(['Unnamed: 0', 'text','hashtag','user_id']+cats, inplace = True, axis =1)

    # readability statistics are nan when the tweet contains no text. so change these scores to 0
    reads = ['ARI','LIX','RIX','complex_words','Coleman-Liau', 'FleschReadingEase', 'GunningFogIndex', 'LIX', 'SMOGIndex', 'RIX', 'DaleChallIndex']
    for r in reads:
        df[[r]] = df[[r]].fillna(value=0).copy()

    cases = list(set(df['target'].tolist()))

    target = df[df['is-target']==1]
    informers = df[df['is-informer']==1]
    infectors = df[df['is-infector']==1]

    n = len(infectors)

    # clean a little
    for sub in [infectors,informers,target]:
        sub.drop(['is-target','is-infector','is-informer'], inplace = True, axis =1)

    cols = infectors.columns.tolist()

    infector_differences = []
    informer_differences = []

    for case_id in cases:

        targ = target.copy()[target['target']==case_id]
        o_fect = infectors.copy()[infectors['target']==case_id]
        o_formers = informers.copy()[informers['target']==case_id]

        targ = targ.copy()[user_cols].to_numpy()
        o_fect = o_fect.copy()[user_cols].to_numpy()
        o_formers = o_formers.copy()[user_cols].to_numpy()

        fect_diff = abs(targ-o_fect)
        former_diff = abs(targ-o_formers)

        
        infector_differences.append(fect_diff.tolist()[0])
        informer_differences.extend(former_diff.tolist())

    
    infector_diff_df = pd.DataFrame(infector_differences)
    infector_diff_df.columns = user_cols
    informer_diff_df = pd.DataFrame( informer_differences )
    informer_diff_df.columns = user_cols

    
    infector_diff_df.insert(0,column = 'label', value= [1]*len(infector_diff_df) )
    informer_diff_df.insert(0,column = 'label', value= [0]*len(informer_diff_df) )
    

    if h_i == 0:
        all_differences = pd.concat([infector_diff_df, informer_diff_df], axis = 0)
    else:   
        all_differences = pd.concat([all_differences, infector_diff_df, informer_diff_df], axis = 0)
    

    print(f'\n\nAdded {hashtag} to DATAFRAME\n----------------------------------------------------------------')

C:\Users\berke\AppData\Local\Temp\ipykernel_27884\3123863060.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub.drop(['is-target','is-infector','is-informer'], inplace = True, axis =1)
C:\Users\berke\AppData\Local\Temp\ipykernel_27884\3123863060.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub.drop(['is-target','is-infector','is-informer'], inplace = True, axis =1)
C:\Users\berke\AppData\Local\Temp\ipykernel_27884\3123863060.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u



Added avengers to DATAFRAME
----------------------------------------------------------------


C:\Users\berke\AppData\Local\Temp\ipykernel_27884\3123863060.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub.drop(['is-target','is-infector','is-informer'], inplace = True, axis =1)
C:\Users\berke\AppData\Local\Temp\ipykernel_27884\3123863060.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub.drop(['is-target','is-infector','is-informer'], inplace = True, axis =1)
C:\Users\berke\AppData\Local\Temp\ipykernel_27884\3123863060.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u



Added blm to DATAFRAME
----------------------------------------------------------------


C:\Users\berke\AppData\Local\Temp\ipykernel_27884\3123863060.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub.drop(['is-target','is-infector','is-informer'], inplace = True, axis =1)
C:\Users\berke\AppData\Local\Temp\ipykernel_27884\3123863060.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub.drop(['is-target','is-infector','is-informer'], inplace = True, axis =1)
C:\Users\berke\AppData\Local\Temp\ipykernel_27884\3123863060.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

...



Added olivianewtonjohn to DATAFRAME
----------------------------------------------------------------


C:\Users\berke\AppData\Local\Temp\ipykernel_27884\3123863060.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub.drop(['is-target','is-infector','is-informer'], inplace = True, axis =1)
C:\Users\berke\AppData\Local\Temp\ipykernel_27884\3123863060.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub.drop(['is-target','is-infector','is-informer'], inplace = True, axis =1)
C:\Users\berke\AppData\Local\Temp\ipykernel_27884\3123863060.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u



Added supercup to DATAFRAME
----------------------------------------------------------------


C:\Users\berke\AppData\Local\Temp\ipykernel_27884\3123863060.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub.drop(['is-target','is-infector','is-informer'], inplace = True, axis =1)
C:\Users\berke\AppData\Local\Temp\ipykernel_27884\3123863060.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub.drop(['is-target','is-infector','is-informer'], inplace = True, axis =1)
C:\Users\berke\AppData\Local\Temp\ipykernel_27884\3123863060.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u



Added UkraineWar to DATAFRAME
----------------------------------------------------------------


In [23]:
all_differences.to_csv('multisource_analysis/difference_scores/difference_database.csv')

### user columns

In [4]:
user_cols = [ 'user_neg_mean',
 'user_neu_mean',
 'user_pos_mean',
 'user_comp_mean',
 'user_arts_&_culture_mean',
 'user_business_&_entrepreneurs_mean',
 'user_celebrity_&_pop_culture_mean',
 'user_diaries_&_daily_life_mean',
 'user_family_mean',
 'user_fashion_&_style_mean',
 'user_film_tv_&_video_mean',
 'user_fitness_&_health_mean',
 'user_food_&_dining_mean',
 'user_gaming_mean',
 'user_learning_&_educational_mean',
 'user_music_mean',
 'user_news_&_social_concern_mean',
 'user_other_hobbies_mean',
 'user_relationships_mean',
 'user_science_&_technology_mean',
 'user_sports_mean',
 'user_travel_&_adventure_mean',
 'user_youth_&_student_life_mean',
 'user_hateful_mean',
 'user_targeted_mean',
 'user_aggressive_mean',
 'user_joy_mean',
 'user_sadness_mean',
 'user_others_mean',
 'user_anger_mean',
 'user_surprise_mean',
 'user_disgust_mean',
 'user_fear_mean',
 'user_topic_mode',
 'user_negative_count',
 'user_neutral_count',
 'user_positive_count',
 'user_arts_&_culture_count',
 'user_business_&_entrepreneurs_count',
 'user_celebrity_&_pop_culture_count',
 'user_diaries_&_daily_life_count',
 'user_family_count',
 'user_fashion_&_style_count',
 'user_film_tv_&_video_count',
 'user_fitness_&_health_count',
 'user_food_&_dining_count',
 'user_gaming_count',
 'user_learning_&_educational_count',
 'user_music_count',
 'user_news_&_social_concern_count',
 'user_other_hobbies_count',
 'user_relationships_count',
 'user_science_&_technology_count',
 'user_sports_count',
 'user_travel_&_adventure_count',
 'user_youth_&_student_life_count',
 'user_politeness_mean',
 'user_polarity_mean',
 'user_subjectivity_mean']

## get the data functions


In [ ]:


for i,col in enumerate(user_cols):

    fec = all_infector_df[col].tolist()
    targ = all_target_df[col].tolist()
    av_form = all_av_informer_df[col].tolist()
    best_form = all_best_informer_df[col].tolist()

    fect_diff = abs(np.array(targ)-np.array(fec))
    inf_diff = abs(np.array(targ)-np.array(best_form))
    
    if i == 0:
        infector_df = pd.DataFrame(fect_diff).T
        best_informer_df = pd.DataFrame()

best_scores_df = pd.DataFrame(best_scores).T
best_scores_df.columns = user_cols
best_scores_df= best_scores_df.copy().T
